# Setup

In [1]:
save_files = True

In [2]:
%%capture
%pip install git+https://github.com/neelnanda-io/TransformerLens.git

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
# import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

import pickle
from google.colab import files

import matplotlib.pyplot as plt
import statistics

In [4]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer #, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [5]:
torch.set_grad_enabled(False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import pdb

## Import functions from repo

In [7]:
!git clone https://github.com/apartresearch/seqcont_circuits.git
%cd /content/seqcont_circuits/src/iter_node_pruning

Cloning into 'seqcont_circuits'...
remote: Enumerating objects: 875, done.
remote: Counting objects: 100% (341/341), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 875 (delta 187), reused 257 (delta 107), pack-reused 534
Receiving objects: 100% (875/875), 16.78 MiB | 14.70 MiB/s, done.
Resolving deltas: 100% (550/550), done.
/content/seqcont_circuits/src/iter_node_pruning


In [8]:
## comment this out when debugging functions in colab to use funcs defined in colab

# don't improt this
# # from dataset import Dataset

from metrics import *
from head_ablation_fns import *
from mlp_ablation_fns import *
from node_ablation_fns import *
from loop_node_ablation_fns import *

## fns

In [9]:
class Dataset:
    def __init__(self, prompts, pos_dict, tokenizer):  # , S1_is_first=False
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)
        self.max_len = max(
            [
                len(self.tokenizer(prompt["text"]).input_ids)
                for prompt in self.prompts
            ]
        )
        all_ids = [0 for prompt in self.prompts] # only 1 template
        all_ids_ar = np.array(all_ids)
        self.groups = []
        for id in list(set(all_ids)):
            self.groups.append(np.where(all_ids_ar == id)[0])

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )
        self.corr_tokenIDs = [
            # self.tokenizer.encode(" " + prompt["corr"])[0] for prompt in self.prompts
            self.tokenizer.encode(prompt["corr"])[0] for prompt in self.prompts
        ]
        self.incorr_tokenIDs = [
            # self.tokenizer.encode(" " + prompt["incorr"])[0] for prompt in self.prompts
            self.tokenizer.encode(prompt["incorr"])[0] for prompt in self.prompts
        ]

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a dict whose values are tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        # for targ in [key for key in self.prompts[0].keys() if (key != 'text' and key != 'corr' and key != 'incorr')]:
        for targ in [key for key in pos_dict]:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = self.tokenizer.tokenize(input_text)
                # if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                #     target_token = prompt[targ]
                # else:
                #     target_token = "Ġ" + prompt[targ]
                # target_index = tokens.index(target_token)
                target_index = pos_dict[targ]
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

In [10]:
def generate_prompts_list_longer(text, tokens):
    prompts_list = []
    prompt_dict = {
        'corr': str(1),
        'incorr': str(2),
        'text': text
        # 'text': model.to_string(tokens)[0]
        }
    tokens_as_strs = model.tokenizer.tokenize(text)
    # tokens_as_strs = model.to_string(tokens)[0].split()
    # for i in range(tokens.shape[1]):
    for i, tok in enumerate(tokens_as_strs):
        prompt_dict['S'+str(i)] = tok
    # for i, tok in enumerate(tokens):
    #     prompt_dict['S'+str(i)] = model.to_string(tok)

    # prompt_dict = {
    #     'corr': '4',
    #     'incorr': '3',
    #     'text': model.to_string(tokens)[0]
    # }
    # # list_tokens = tokenizer.tokenize('1 2 3 ')
    # tokens_as_strs = model.to_string(tokens)[0].split()
    # for i, tok_as_str in enumerate(tokens_as_strs):
    #     if tok_as_str == '▁':
    #         prompt_dict['S'+str(i)] = ' '
    #     else:
    #         prompt_dict['S'+str(i)] = tok_as_str
    prompts_list.append(prompt_dict)
    return prompts_list

# Load Model

In [11]:
from transformers import LlamaForCausalLM, LlamaTokenizer

In [12]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [13]:
LLAMA_2_7B_CHAT_PATH = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(LLAMA_2_7B_CHAT_PATH)
# tokenizer = LlamaTokenizer.from_pretrained(LLAMA_2_7B_CHAT_PATH, use_fast= False, add_prefix_space= False)
hf_model = LlamaForCausalLM.from_pretrained(LLAMA_2_7B_CHAT_PATH, low_cpu_mem_usage=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [14]:
import transformer_lens.utils as utils
from transformer_lens.hook_points import HookPoint
from transformer_lens import HookedTransformer

In [15]:
model = HookedTransformer.from_pretrained(
    LLAMA_2_7B_CHAT_PATH,
    hf_model = hf_model,
    tokenizer = tokenizer,
    device = "cpu",
    fold_ln = False,
    center_writing_weights = False,
    center_unembed = False,
)

del hf_model

model = model.to("cuda" if torch.cuda.is_available() else "cpu")

Loaded pretrained model meta-llama/Llama-2-7b-chat-hf into HookedTransformer
Moving model to device:  cuda


# new functions

In [16]:
# class Dataset:
#     def __init__(self, prompts, pos_dict, tokenizer, tokens):  # , S1_is_first=False
#         self.prompts = prompts
#         self.tokenizer = tokenizer
#         self.N = len(prompts)
#         self.max_len = max(
#             [
#                 len(self.tokenizer(prompt["text"]).input_ids)
#                 for prompt in self.prompts
#             ]
#         )
#         all_ids = [0 for prompt in self.prompts] # only 1 template
#         all_ids_ar = np.array(all_ids)
#         self.groups = []
#         for id in list(set(all_ids)):
#             self.groups.append(np.where(all_ids_ar == id)[0])

#         texts = [ prompt["text"] for prompt in self.prompts ]
#         # self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
#         #     torch.int
#         # )
#         self.toks = tokens
#         self.corr_tokenIDs = [
#             # self.tokenizer.encode(" " + prompt["corr"])[0] for prompt in self.prompts
#             self.tokenizer.encode(prompt["corr"])[0] for prompt in self.prompts
#         ]
#         self.incorr_tokenIDs = [
#             # self.tokenizer.encode(" " + prompt["incorr"])[0] for prompt in self.prompts
#             self.tokenizer.encode(prompt["incorr"])[0] for prompt in self.prompts
#         ]

#         # word_idx: for every prompt, find the token index of each target token and "end"
#         # word_idx is a dict whose values are tensor with an element for each prompt. The element is the targ token's ind at that prompt
#         self.word_idx = {}
#         # for targ in [key for key in self.prompts[0].keys() if (key != 'text' and key != 'corr' and key != 'incorr')]:
#         for targ in [key for key in pos_dict]:
#             targ_lst = []
#             for prompt in self.prompts:
#                 input_text = prompt["text"]
#                 # tokens = self.tokenizer.tokenize(input_text)
#                 # if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
#                 #     target_token = prompt[targ]
#                 # else:
#                 #     target_token = "Ġ" + prompt[targ]
#                 # target_index = tokens.index(target_token)
#                 target_index = pos_dict[targ]
#                 targ_lst.append(target_index)
#             self.word_idx[targ] = torch.tensor(targ_lst)

#         # targ_lst = []
#         # for prompt in self.prompts:
#         #     input_text = prompt["text"]
#         #     tokens = self.tokenizer.tokenize(input_text)
#         #     end_token_index = len(tokens) - 1
#         #     targ_lst.append(end_token_index)
#         # self.word_idx["end"] = torch.tensor(targ_lst)

#     def __len__(self):
#         return self.N

In [17]:
def get_heads_actv_mean(
    means_dataset: Dataset,
    model: HookedTransformer
) -> Float[Tensor, "layer batch seq head_idx d_head"]:
    '''
    Output: The mean activations of a head's output
    '''
    _, means_cache = model.run_with_cache(
        means_dataset.toks.long(),
        return_type=None,
        names_filter=lambda name: name.endswith("z"),
    )
    n_layers, n_heads, d_head = model.cfg.n_layers, model.cfg.n_heads, model.cfg.d_head
    batch, seq_len = len(means_dataset), means_dataset.max_len
    means = t.zeros(size=(n_layers, batch, seq_len, n_heads, d_head), device=model.cfg.device)

    # for layer in range(model.cfg.n_layers):
    #     z_for_this_layer: Float[Tensor, "batch seq head d_head"] = means_cache[utils.get_act_name("z", layer)]
    #     for template_group in means_dataset.groups:
    #         z_for_this_template = z_for_this_layer[template_group]
    #         z_means_for_this_template = einops.reduce(z_for_this_template, "batch seq head d_head -> seq head d_head", "mean")
    #         if z_means_for_this_template.shape[0] == 5:
    #             pdb.set_trace()
    #         means[layer, template_group] = z_means_for_this_template

    del(means_cache)

    return means

In [18]:
# def mask_circ_heads(
#     means_dataset: Dataset,
#     model: HookedTransformer,
#     circuit: Dict[str, List[Tuple[int, int]]],
#     seq_pos_to_keep: Dict[str, str],
# ) -> Dict[int, Bool[Tensor, "batch seq head"]]:
#     '''
#     Output: for each layer, a mask of circuit components that should not be ablated
#     '''
#     heads_and_posns_to_keep = {}
#     batch, seq, n_heads = len(means_dataset), means_dataset.max_len, model.cfg.n_heads

#     for layer in range(model.cfg.n_layers):

#         mask = t.zeros(size=(batch, seq, n_heads))

#         for (head_type, head_list) in circuit.items():
#             seq_pos = seq_pos_to_keep[head_type]
#             # if seq_pos == 'S7':
#             #     pdb.set_trace()
#             indices = means_dataset.word_idx[seq_pos] # modify this for key vs query pos. curr, this is query
#             for (layer_idx, head_idx) in head_list:
#                 if layer_idx == layer:
#                     # if indices.item() == 7:
#                     #     pdb.set_trace()
#                     mask[:, indices, head_idx] = 1
#                     # mask[:, :, head_idx] = 1  # keep L.H at all pos

#         heads_and_posns_to_keep[layer] = mask.bool()
#     # pdb.set_trace()
#     return heads_and_posns_to_keep

In [19]:
def mask_circ_heads(
    means_dataset: Dataset,
    model: HookedTransformer,
    circuit: Dict[str, List[Tuple[int, int]]],
    seq_pos_to_keep: Dict[str, str],
) -> Dict[int, Bool[Tensor, "batch seq head"]]:
    '''
    Output: for each layer, a mask of circuit components that should not be ablated
    '''
    heads_and_posns_to_keep = {}
    # batch, seq, n_heads = len(means_dataset), means_dataset.max_len, model.cfg.n_heads
    batch, seq, n_heads = len(means_dataset), len(circuit.keys()), model.cfg.n_heads
    print(seq)

    for layer in range(model.cfg.n_layers):

        mask = t.zeros(size=(batch, seq, n_heads))

        for (head_type, head_list) in circuit.items():
            seq_pos = seq_pos_to_keep[head_type]
            indices = means_dataset.word_idx[seq_pos] # modify this for key vs query pos. curr, this is query
            for (layer_idx, head_idx) in head_list:
                if layer_idx == layer:
                    # mask[:, indices, head_idx] = 1
                    mask[:, :, head_idx] = 1

        heads_and_posns_to_keep[layer] = mask.bool()

    return heads_and_posns_to_keep

In [20]:
def hook_func_mask_head(
    z: Float[Tensor, "batch seq head d_head"],
    hook: HookPoint,
    # components_to_keep: Dict[int, Bool[Tensor, "batch seq head"]],
    # means: Float[Tensor, "layer batch seq head d_head"],
    circuit: Dict[str, List[Tuple[int, int]]],
) -> Float[Tensor, "batch seq head d_head"]:
    '''
    Use this to not mask components
    '''
    # mask_for_this_layer = components_to_keep[hook.layer()].unsqueeze(-1).to(z.device)
    # z = t.where(mask_for_this_layer, z, means[hook.layer()])

    ###
    # heads_and_posns_to_keep = {}
    # batch, seq, n_heads = z.shape[0], z.shape[1], model.cfg.n_heads  # components_to_keep[0].shape[0] is batch

    # for layer in range(model.cfg.n_layers):

    #     mask = t.zeros(size=(batch, seq, n_heads))

    #     for (head_type, head_list) in circuit.items():
    #         # seq_pos = seq_pos_to_keep[head_type]
    #         # indices = means_dataset.word_idx[seq_pos] # modify this for key vs query pos. curr, this is query
    #         for (layer_idx, head_idx) in head_list:
    #             if layer_idx == layer:
    #                 # mask[:, indices, head_idx] = 1
    #                 mask[:, :, head_idx] = 1

    #     heads_and_posns_to_keep[layer] = mask.bool()
    ###
    mask_for_this_layer = t.zeros(size=(z.shape[0], z.shape[1], z.shape[2]))
    for (head_type, head_list) in circuit.items():
        # seq_pos = seq_pos_to_keep[head_type]
        # indices = means_dataset.word_idx[seq_pos] # modify this for key vs query pos. curr, this is query
        for (layer_idx, head_idx) in head_list:
            if layer_idx == hook.layer():
                # mask[:, indices, head_idx] = 1
                mask_for_this_layer[:, :, head_idx] = 1

    mask_for_this_layer = mask_for_this_layer.bool()
    mask_for_this_layer = mask_for_this_layer.unsqueeze(-1).to(z.device)  # d_model is 1; then is broadcast in where

    z = t.where(mask_for_this_layer, z, 0)

    return z

In [21]:
def add_ablation_hook_head(
    model: HookedTransformer,
    means_dataset: Dataset,
    circuit: Dict[str, List[Tuple[int, int]]],
    seq_pos_to_keep: Dict[str, str],
    is_permanent: bool = True,
) -> HookedTransformer:
    '''
    Ablate the model, except as components and positions to keep
    '''

    model.reset_hooks(including_permanent=True)
    means = get_heads_actv_mean(means_dataset, model)
    components_to_keep = mask_circ_heads(means_dataset, model, circuit, seq_pos_to_keep)

    hook_fn = partial(
        hook_func_mask_head,
        # components_to_keep=components_to_keep,
        # means=means,
        circuit=circuit,
    )

    model.add_hook(lambda name: name.endswith("z"), hook_fn, is_permanent=is_permanent)
    return model

In [22]:
# from dataset import Dataset
from transformer_lens import HookedTransformer, utils
from transformer_lens.hook_points import HookPoint
import einops
from functools import partial
import torch as t
from torch import Tensor
from typing import Dict, Tuple, List
from jaxtyping import Float, Bool

# from head_ablation_fns import *
# from mlp_ablation_fns import *

def add_ablation_hook_MLP_head(
    model: HookedTransformer,
    means_dataset: Dataset,
    heads_lst, mlp_lst,
    is_permanent: bool = True,
) -> HookedTransformer:
    CIRCUIT = {}
    SEQ_POS_TO_KEEP = {}
    # for i in range(len(model.tokenizer.tokenize(means_dataset.prompts[0]['text']))):
    num_pos = len(model.tokenizer(means_dataset.prompts[0]['text']).input_ids)
    for i in range(num_pos ):
        CIRCUIT['S'+str(i)] = heads_lst
        # if i == len(model.tokenizer.tokenize(means_dataset.prompts[0]['text'])) - 1:
        # if i == num_pos - 1:
        #     SEQ_POS_TO_KEEP['S'+str(i)] = 'end'
        # else:
        SEQ_POS_TO_KEEP['S'+str(i)] = 'S'+str(i)

    model.reset_hooks(including_permanent=True)

    # Compute the mean of each head's output on the ABC dataset, grouped by template
    means = get_heads_actv_mean(means_dataset, model)
    # Convert this into a boolean map
    components_to_keep = mask_circ_heads(means_dataset, model, CIRCUIT, SEQ_POS_TO_KEEP)

    # Get a hook function which will patch in the mean z values for each head, at
    # all positions which aren't important for the circuit
    hook_fn = partial(
        hook_func_mask_head,
        # components_to_keep=components_to_keep,
        # means=means,
        circuit=CIRCUIT,
    )

    # Apply hook
    model.add_hook(lambda name: name.endswith("z"), hook_fn, is_permanent=is_permanent)

    # if all_entries_true(components_to_keep) == False:
    #     pdb.set_trace()
    ########################
    # CIRCUIT = {}
    # SEQ_POS_TO_KEEP = {}
    # # for i in range(len(model.tokenizer.tokenize(means_dataset.prompts[0]['text']))):
    # num_pos = len(model.tokenizer(means_dataset.prompts[0]['text']).input_ids)
    # for i in range(num_pos ):
    #     CIRCUIT['S'+str(i)] = mlp_lst
    #     # if i == len(model.tokenizer.tokenize(means_dataset.prompts[0]['text'])) - 1:
    #     # if i == num_pos - 1:
    #     #     SEQ_POS_TO_KEEP['S'+str(i)] = 'end'
    #     # else:
    #     SEQ_POS_TO_KEEP['S'+str(i)] = 'S'+str(i)

    # # Compute the mean of each head's output on the ABC dataset, grouped by template
    # means = get_MLPs_actv_mean(means_dataset, model)

    # # Convert this into a boolean map
    # components_to_keep = mask_circ_MLPs(means_dataset, model, CIRCUIT, SEQ_POS_TO_KEEP)

    # # Get a hook function which will patch in the mean z values for each head, at
    # # all positions which aren't important for the circuit
    # hook_fn = partial(
    #     hook_func_mask_mlp_out,
    #     components_to_keep=components_to_keep,
    #     means=means
    # )

    # model.add_hook(lambda name: name.endswith("mlp_out"), hook_fn, is_permanent=True)

    return model

In [23]:
def all_entries_true(tensor_dict):
    for key, tensor in tensor_dict.items():
        if not torch.all(tensor).item():
            return False
    return True

# ablation fns mult tok answers

In [24]:
def clean_gen(model, clean_text, corr_ans):
    model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
    tokens = model.to_tokens(clean_text).to(device)
    # tokens = tokens[:, 1:] # get rid of prepend bos when using model.to_tokens

    total_score = 0
    corr_ans_tokLen = 0
    ans_so_far = ''
    # while True:
    for i in range(5):
        print(f"Sequence so far: {model.to_string(tokens)[0]!r}")
        logits = model(tokens)
        next_token = logits[0, -1].argmax(dim=-1) # Get the predicted token at the end of our sequence
        next_char = model.to_string(next_token)

        corr_logits = logits[:, -1, next_token]
        total_score += corr_logits
        print(f"logit diff of new char: {corr_logits}")

        ans_so_far += next_char
        corr_ans_tokLen += 1
        print(f"{tokens.shape[-1]+1}th char = {next_char!r}")
        if ans_so_far == corr_ans:
            print('\nTotal logit diff: ', total_score.item())
            break

        # Define new input sequence, by appending the previously generated token
        tokens = torch.cat([tokens, next_token[None, None]], dim=-1)
        # if next_char == '':
        #     next_char = ' '
        # clean_text = clean_text + next_char
        # tokens = model.to_tokens(clean_text).to(device)
    return corr_ans_tokLen

In [25]:
clean_text = "1 2 3"
corr_ans = ' 5'
corr_ans_tokLen = clean_gen(model, clean_text, corr_ans)

Sequence so far: '<s> 1 2 3'
logit diff of new char: tensor([16.3678], device='cuda:0')
8th char = ''
Sequence so far: '<s> 1 2 3 '
logit diff of new char: tensor([19.4441], device='cuda:0')
9th char = '4'
Sequence so far: '<s> 1 2 3 4'
logit diff of new char: tensor([20.5775], device='cuda:0')
10th char = ''
Sequence so far: '<s> 1 2 3 4 '
logit diff of new char: tensor([19.2083], device='cuda:0')
11th char = '5'
Sequence so far: '<s> 1 2 3 4 5'
logit diff of new char: tensor([18.8848], device='cuda:0')
12th char = ''


In [26]:
def ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, corr_ans_tokLen):
    tokens = model.to_tokens(clean_text).to(device)
    prompts_list = generate_prompts_list_longer(clean_text, tokens)

    corr_tokens = model.to_tokens(corr_text).to(device)
    prompts_list_2 = generate_prompts_list_longer(corr_text, corr_tokens)

    model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
    pos_dict = {}
    # for i in range(len(model.tokenizer.tokenize(prompts_list_2[0]['text']))):
    num_pos = len(model.tokenizer(prompts_list_2[0]['text']).input_ids)
    # for i in range(num_pos + 1):
    for i in range(num_pos ):
        pos_dict['S'+str(i)] = i
    dataset_2 = Dataset(prompts_list_2, pos_dict, model.tokenizer)
    # dataset_2 = Dataset(prompts_list_2, pos_dict, model.tokenizer, tokens)
    # pdb.set_trace()
    model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

    # tokens = tokens[:, 1:] # get rid of prepend bos when using model.to_tokens
    logits = model(tokens)
    next_token = logits[0, -1].argmax(dim=-1)
    next_char = model.to_string(next_token)

    total_score = 0

    for i in range(corr_ans_tokLen):
    # for i in range(5):
        if next_char == '':
            next_char = ' '

        clean_text = clean_text + next_char
        # tokens = model.to_tokens(clean_text).to(device)
        # tokens = tokens[:, 1:]
        print(f"Sequence so far: {clean_text}")
        print(f"{tokens.shape[-1]+1}th char = {next_char!r}")

        # clean_text = model.to_string(tokens)[0]
        tokens = torch.cat([tokens, next_token[None, None]], dim=-1)
        # print(clean_text)
        # print(tokens.shape)

        # get new ablation dataset
        # model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

        # corr_text = corr_text + next_char
        # # corr_tokens = model.to_tokens(corr_text).to(device)

        # # corr_text = model.to_string(corr_tokens)[0]
        # corr_tokens = torch.cat([corr_tokens, next_token[None, None]], dim=-1)
        # prompts_list_2 = generate_prompts_list_longer(corr_text, corr_tokens)
        # print(corr_text)
        # # print(corr_tokens.shape)

        # pos_dict = {}
        # # for i in range(len(model.tokenizer.tokenize(prompts_list_2[0]['text']))):
        # # for i in range(corr_tokens.shape[1]):
        # num_pos = len(model.tokenizer(prompts_list_2[0]['text']).input_ids)
        # # for i in range(num_pos + 1):
        # for i in range(num_pos ):
        #     pos_dict['S'+str(i)] = i

        # # dataset_2 = Dataset(prompts_list_2, pos_dict, model.tokenizer)
        # dataset_2 = Dataset(prompts_list_2, pos_dict, model.tokenizer, corr_tokens)

        # model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

        logits = model(tokens)
        next_token = logits[0, -1].argmax(dim=-1) # Get the predicted token at the end of our sequence
        next_char = model.to_string(next_token)

        # print('\n')
        # print(f"Sequence so far: {model.to_string(tokens)[0]!r}")

        # new_score = get_logit_diff(logits, dataset)
        # total_score += new_score
        # print(f"corr logit of new char: {new_score}")
    # print('\n Total corr logit: ', total_score.item())

# test clean prompts

In [26]:
# clean_text = "1"
# tokens = model.to_tokens(clean_text).to(device)
# tokens = tokens[:, 1:] # get rid of prepend bos when using model.to_tokens
# # tokens = model.tokenizer(clean_text)['input_ids']
# logits = model(tokens)
# next_token = logits[0, -1].argmax(dim=-1) # Get the predicted token at the end of our sequence
# next_char = model.to_string(next_token)

clean_text = "1"
corr_ans_tokLen = 3
clean_gen(model, clean_text, corr_ans)

Sequence so far: '<s> 1'
logit diff of new char: tensor([12.9576], device='cuda:0')
4th char = '.'
Sequence so far: '<s> 1.'
logit diff of new char: tensor([5.0549], device='cuda:0')
5th char = ''
Sequence so far: '<s> 1. '
logit diff of new char: tensor([8.5630], device='cuda:0')
6th char = 'Introduction'
Sequence so far: '<s> 1.  Introduction'
logit diff of new char: tensor([19.3782], device='cuda:0')
7th char = '<0x0A>'
Sequence so far: '<s> 1.  Introduction\n'
logit diff of new char: tensor([12.2388], device='cuda:0')
8th char = '<0x0A>'


5

In [27]:
clean_text = "two"
corr_ans_tokLen = 3
clean_gen(model, clean_text, corr_ans)

Sequence so far: '<s> two'
logit diff of new char: tensor([5.4994], device='cuda:0')
3th char = '-'
Sequence so far: '<s> two-'
logit diff of new char: tensor([10.8123], device='cuda:0')
4th char = 'time'
Sequence so far: '<s> two-time'
logit diff of new char: tensor([10.4234], device='cuda:0')
5th char = 'Pul'
Sequence so far: '<s> two-time Pul'
logit diff of new char: tensor([24.6146], device='cuda:0')
6th char = 'itzer'
Sequence so far: '<s> two-time Pulitzer'
logit diff of new char: tensor([22.3427], device='cuda:0')
7th char = 'Prize'


5

In [28]:
clean_text = "March"
corr_ans_tokLen = 3
clean_gen(model, clean_text, corr_ans)

Sequence so far: '<s> March'
logit diff of new char: tensor([17.5723], device='cuda:0')
3th char = ''
Sequence so far: '<s> March '
logit diff of new char: tensor([19.5974], device='cuda:0')
4th char = '2'
Sequence so far: '<s> March 2'
logit diff of new char: tensor([18.3477], device='cuda:0')
5th char = '0'
Sequence so far: '<s> March 20'
logit diff of new char: tensor([20.1788], device='cuda:0')
6th char = '1'
Sequence so far: '<s> March 201'
logit diff of new char: tensor([19.7587], device='cuda:0')
7th char = '7'


5

In [29]:
clean_text = "Bob is first. David is"
corr_ans_tokLen = 3
clean_gen(model, clean_text, corr_ans)

Sequence so far: '<s> Bob is first. David is'
logit diff of new char: tensor([13.4227], device='cuda:0')
8th char = 'second'
Sequence so far: '<s> Bob is first. David is second'
logit diff of new char: tensor([20.7910], device='cuda:0')
9th char = '.'
Sequence so far: '<s> Bob is first. David is second.'
logit diff of new char: tensor([12.5447], device='cuda:0')
10th char = '<0x0A>'
Sequence so far: '<s> Bob is first. David is second.\n'
logit diff of new char: tensor([11.2046], device='cuda:0')
11th char = '<0x0A>'
Sequence so far: '<s> Bob is first. David is second.\n\n'
logit diff of new char: tensor([11.9983], device='cuda:0')
12th char = 'Bob'


5

In [30]:
clean_text = "Two days after Monday is"
corr_ans_tokLen = 3
clean_gen(model, clean_text, corr_ans)

Sequence so far: '<s> Two days after Monday is'
logit diff of new char: tensor([17.0134], device='cuda:0')
7th char = 'T'
Sequence so far: '<s> Two days after Monday is T'
logit diff of new char: tensor([22.0867], device='cuda:0')
8th char = 'ues'
Sequence so far: '<s> Two days after Monday is Tues'
logit diff of new char: tensor([27.3643], device='cuda:0')
9th char = 'day'
Sequence so far: '<s> Two days after Monday is Tuesday'
logit diff of new char: tensor([18.9252], device='cuda:0')
10th char = '.'
Sequence so far: '<s> Two days after Monday is Tuesday.'
logit diff of new char: tensor([10.3426], device='cuda:0')
11th char = '<0x0A>'


5

In [31]:
clean_text = "uno dos tres"
corr_ans_tokLen = 3
clean_gen(model, clean_text, corr_ans)

Sequence so far: '<s> uno dos tres'
logit diff of new char: tensor([7.8090], device='cuda:0')
5th char = 'cuatro'
Sequence so far: '<s> uno dos tres cuatro'
logit diff of new char: tensor([9.7915], device='cuda:0')
6th char = 'cinco'
Sequence so far: '<s> uno dos tres cuatro cinco'
logit diff of new char: tensor([9.3842], device='cuda:0')
7th char = 'seis'
Sequence so far: '<s> uno dos tres cuatro cinco seis'
logit diff of new char: tensor([12.1068], device='cuda:0')
8th char = 'seven'
Sequence so far: '<s> uno dos tres cuatro cinco seis seven'
logit diff of new char: tensor([16.4966], device='cuda:0')
9th char = 'eight'


5

In [32]:
clean_text = "one two three"
corr_ans_tokLen = 3
clean_gen(model, clean_text, corr_ans)

Sequence so far: '<s> one two three'
logit diff of new char: tensor([10.4469], device='cuda:0')
5th char = 'four'
Sequence so far: '<s> one two three four'
logit diff of new char: tensor([16.2546], device='cuda:0')
6th char = 'five'
Sequence so far: '<s> one two three four five'
logit diff of new char: tensor([15.9061], device='cuda:0')
7th char = 'six'
Sequence so far: '<s> one two three four five six'
logit diff of new char: tensor([18.5995], device='cuda:0')
8th char = 'seven'
Sequence so far: '<s> one two three four five six seven'
logit diff of new char: tensor([18.7767], device='cuda:0')
9th char = 'eight'


5

In [33]:
clean_text = "2 4 6 "
corr_ans_tokLen = 3
clean_gen(model, clean_text, corr_ans)

Sequence so far: '<s> 2 4 6 '
logit diff of new char: tensor([19.3199], device='cuda:0')
9th char = '8'
Sequence so far: '<s> 2 4 6 8'
logit diff of new char: tensor([9.7377], device='cuda:0')
10th char = ''
Sequence so far: '<s> 2 4 6 8 '
logit diff of new char: tensor([16.1787], device='cuda:0')
11th char = '1'
Sequence so far: '<s> 2 4 6 8 1'
logit diff of new char: tensor([22.3343], device='cuda:0')
12th char = '0'
Sequence so far: '<s> 2 4 6 8 10'
logit diff of new char: tensor([7.2428], device='cuda:0')
13th char = ''


5

# 1 2 3 genr ablation expms

In [72]:
clean_text = "1 2 3"
corr_text = "5 3 9"
# corr_text = "1 2 3"

clean

In [44]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
mlps_not_ablate = [layer for layer in range(32)]
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 1 2 3 
9th char = ' '
Sequence so far: 1 2 3 4
10th char = '4'
Sequence so far: 1 2 3 4 
11th char = ' '


ablate all

In [45]:
clean_text = "1 2 3"
corr_text = "5 3 9"
heads_not_ablate = []  # ablate all heads but not MLPs
mlps_not_ablate = []  # ablate all MLPs
corr_ans_tokLen = 2
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, corr_ans_tokLen)

7
Sequence so far: 1 2 39
9th char = '9'
Sequence so far: 1 2 399
10th char = '9'


ablate just head 20.7

In [46]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate.remove((20, 7))

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 1 2 3 
9th char = ' '
Sequence so far: 1 2 3 4
10th char = '4'
Sequence so far: 1 2 3 4 
11th char = ' '


In [47]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 7), (1, 11)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 1 2 3 
9th char = ' '
Sequence so far: 1 2 3 4
10th char = '4'
Sequence so far: 1 2 3 4 
11th char = ' '


In [48]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 7), (1, 11)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32) if layer != 1]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 1 2 3 
9th char = ' '
Sequence so far: 1 2 3 4
10th char = '4'
Sequence so far: 1 2 3 4 
11th char = ' '


In [49]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 7), (1, 11), (16,0)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32) if layer != 1]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 1 2 3 
9th char = ' '
Sequence so far: 1 2 3 4
10th char = '4'
Sequence so far: 1 2 3 4 
11th char = ' '


In [50]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 7), (1, 11), (16,0)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32) if layer <10]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 1 2 3 
9th char = ' '
Sequence so far: 1 2 3 4
10th char = '4'
Sequence so far: 1 2 3 4 
11th char = ' '


In [51]:
heads_not_ablate = []

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 1 2 39
9th char = '9'
Sequence so far: 1 2 399
10th char = '9'
Sequence so far: 1 2 3999
11th char = '9'


In [52]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 7), (1, 11), (16,0), (0, 30), (0, 9), (15,25)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 1 2 3 
9th char = ' '
Sequence so far: 1 2 3 4
10th char = '4'
Sequence so far: 1 2 3 4 
11th char = ' '


## top 20

In [75]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove_num = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove_months = ([(23, 17), (17, 11), (16, 0), (26, 14), (18, 9), (5, 25), (22, 20), (6, 24), (26, 9), (12, 18), (13, 20), (19, 12), (27, 29), (13, 14), (16, 14), (12, 26), (19, 30), (16, 18), (31, 27), (26, 28), (16, 1), (18, 1), (19, 28), (18, 31), (29, 4), (17, 0), (14, 1), (17, 12), (12, 15), (28, 16), (10, 1), (16, 19), (9, 27), (30, 1), (19, 27), (0, 3), (15, 11), (21, 3), (11, 19), (12, 0), (23, 11), (8, 14), (16, 8), (22, 13), (13, 3), (4, 19), (14, 15), (12, 20), (19, 16), (18, 5)])
head_to_remove_num = head_to_remove_num[:20]
head_to_remove_months = head_to_remove_months[:20]
head_to_remove = head_to_remove_num + head_to_remove_months
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 10)

7
Sequence so far: 1 2 3 
8th char = ' '
Sequence so far: 1 2 3 4
9th char = '4'
Sequence so far: 1 2 3 4 
10th char = ' '
Sequence so far: 1 2 3 4 5
11th char = '5'
Sequence so far: 1 2 3 4 5 
12th char = ' '
Sequence so far: 1 2 3 4 5 6
13th char = '6'
Sequence so far: 1 2 3 4 5 6 
14th char = ' '
Sequence so far: 1 2 3 4 5 6 7
15th char = '7'
Sequence so far: 1 2 3 4 5 6 7 
16th char = ' '
Sequence so far: 1 2 3 4 5 6 7 8
17th char = '8'


## top 50

In [73]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 1 2 3 
8th char = ' '
Sequence so far: 1 2 3 4
9th char = '4'
Sequence so far: 1 2 3 4 
10th char = ' '


# 2 4 6

In [101]:
clean_text = "2 4 6"
corr_text = "5 3 9"
# corr_text = "1 2 3"

clean

In [54]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
mlps_not_ablate = [layer for layer in range(32)]
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 2 4 6 
9th char = ' '
Sequence so far: 2 4 6 8
10th char = '8'
Sequence so far: 2 4 6 8 
11th char = ' '


In [55]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(0, 13)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 2 4 6 
9th char = ' '
Sequence so far: 2 4 6 8
10th char = '8'
Sequence so far: 2 4 6 88
11th char = '8'


In [56]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(0, 1)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 2 4 6 
9th char = ' '
Sequence so far: 2 4 6 8
10th char = '8'
Sequence so far: 2 4 6 8 
11th char = ' '


In [57]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(1, 14)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 2 4 6 
9th char = ' '
Sequence so far: 2 4 6 8
10th char = '8'
Sequence so far: 2 4 6 8 
11th char = ' '


## top 50

In [77]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 2 4 6 
8th char = ' '
Sequence so far: 2 4 6 7
9th char = '7'
Sequence so far: 2 4 6 7 
10th char = ' '


In [82]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:20]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 2 4 6 
8th char = ' '
Sequence so far: 2 4 6 8
9th char = '8'
Sequence so far: 2 4 6 8 
10th char = ' '


In [83]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:30]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 2 4 6 
8th char = ' '
Sequence so far: 2 4 6 8
9th char = '8'
Sequence so far: 2 4 6 8 
10th char = ' '


In [84]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:40]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 2 4 6 
8th char = ' '
Sequence so far: 2 4 6 7
9th char = '7'
Sequence so far: 2 4 6 7 
10th char = ' '


In [85]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[30:40]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 2 4 6 
8th char = ' '
Sequence so far: 2 4 6 8
9th char = '8'
Sequence so far: 2 4 6 8 
10th char = ' '


## random

In [79]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 5)

7
Sequence so far: 2 4 6 
8th char = ' '
Sequence so far: 2 4 6 8
9th char = '8'
Sequence so far: 2 4 6 8 
10th char = ' '
Sequence so far: 2 4 6 8 1
11th char = '1'
Sequence so far: 2 4 6 8 10
12th char = '0'


In [80]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 5)

7
Sequence so far: 2 4 6 
8th char = ' '
Sequence so far: 2 4 6 8
9th char = '8'
Sequence so far: 2 4 6 8 
10th char = ' '
Sequence so far: 2 4 6 8 1
11th char = '1'
Sequence so far: 2 4 6 8 10
12th char = '0'


In [81]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 5)

7
Sequence so far: 2 4 6 
8th char = ' '
Sequence so far: 2 4 6 8
9th char = '8'
Sequence so far: 2 4 6 8 
10th char = ' '
Sequence so far: 2 4 6 8 1
11th char = '1'
Sequence so far: 2 4 6 8 10
12th char = '0'


## top 40 from 2 4 6 circ

In [104]:
# from 2 4 6

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
# head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = ([(0, 13), (1, 8), (0, 15), (0, 14), (4, 3), (16, 0), (15, 25), (5, 25), (5, 26), (7, 30), (1, 11), (8, 0), (22, 25), (6, 11), (5, 29), (2, 2), (6, 26), (6, 24), (5, 15), (20, 17), (6, 5), (5, 17), (2, 30), (7, 9), (4, 0), (13, 6), (5, 11), (0, 21), (0, 7), (29, 1), (0, 1), (29, 5), (8, 4), (5, 16), (31, 4), (18, 19), (28, 16), (18, 9), (0, 4), (4, 16)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 7)

7
Sequence so far: 2 4 6 
8th char = ' '
Sequence so far: 2 4 6 8
9th char = '8'
Sequence so far: 2 4 6 8 
10th char = ' '
Sequence so far: 2 4 6 8 1
11th char = '1'
Sequence so far: 2 4 6 8 10
12th char = '0'
Sequence so far: 2 4 6 8 10 
13th char = ' '
Sequence so far: 2 4 6 8 10 1
14th char = '1'


# 10 12 14

In [105]:
clean_text = "10 12 14"
corr_text = "5 3 9"
# corr_text = "1 2 3"

clean

In [87]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
mlps_not_ablate = [layer for layer in range(32)]
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 10 12 14 
11th char = ' '
Sequence so far: 10 12 14 1
12th char = '1'
Sequence so far: 10 12 14 16
13th char = '6'


## top 50

In [88]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 10 12 14 
11th char = ' '
Sequence so far: 10 12 14 1
12th char = '1'
Sequence so far: 10 12 14 10
13th char = '0'


In [91]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:40]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 10 12 14 
11th char = ' '
Sequence so far: 10 12 14 1
12th char = '1'
Sequence so far: 10 12 14 15
13th char = '5'


## random

In [89]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 5)

7
Sequence so far: 10 12 14 
11th char = ' '
Sequence so far: 10 12 14 1
12th char = '1'
Sequence so far: 10 12 14 16
13th char = '6'
Sequence so far: 10 12 14 16 
14th char = ' '
Sequence so far: 10 12 14 16 1
15th char = '1'


In [90]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 5)

7
Sequence so far: 10 12 14 
11th char = ' '
Sequence so far: 10 12 14 1
12th char = '1'
Sequence so far: 10 12 14 16
13th char = '6'
Sequence so far: 10 12 14 16 
14th char = ' '
Sequence so far: 10 12 14 16 1
15th char = '1'


## top 40 from 2 4 6 circ

In [106]:
# from 2 4 6

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
# head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = ([(0, 13), (1, 8), (0, 15), (0, 14), (4, 3), (16, 0), (15, 25), (5, 25), (5, 26), (7, 30), (1, 11), (8, 0), (22, 25), (6, 11), (5, 29), (2, 2), (6, 26), (6, 24), (5, 15), (20, 17), (6, 5), (5, 17), (2, 30), (7, 9), (4, 0), (13, 6), (5, 11), (0, 21), (0, 7), (29, 1), (0, 1), (29, 5), (8, 4), (5, 16), (31, 4), (18, 19), (28, 16), (18, 9), (0, 4), (4, 16)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 7)

7
Sequence so far: 10 12 14 
11th char = ' '
Sequence so far: 10 12 14 1
12th char = '1'
Sequence so far: 10 12 14 16
13th char = '6'
Sequence so far: 10 12 14 16 
14th char = ' '
Sequence so far: 10 12 14 16 1
15th char = '1'
Sequence so far: 10 12 14 16 18
16th char = '8'
Sequence so far: 10 12 14 16 18 
17th char = ' '


# 21 23 25

In [92]:
clean_text = "21 23 25"
corr_text = "5 3 9"
# corr_text = "1 2 3"

clean

In [93]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
mlps_not_ablate = [layer for layer in range(32)]
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 21 23 25 
11th char = ' '
Sequence so far: 21 23 25 2
12th char = '2'
Sequence so far: 21 23 25 27
13th char = '7'


## top 50

In [94]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 21 23 25 
11th char = ' '
Sequence so far: 21 23 25 2
12th char = '2'
Sequence so far: 21 23 25 20
13th char = '0'


In [95]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:40]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 21 23 25 
11th char = ' '
Sequence so far: 21 23 25 2
12th char = '2'
Sequence so far: 21 23 25 22
13th char = '2'


## random

In [98]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 10)

7
Sequence so far: 21 23 25 
11th char = ' '
Sequence so far: 21 23 25 2
12th char = '2'
Sequence so far: 21 23 25 27
13th char = '7'
Sequence so far: 21 23 25 27 
14th char = ' '
Sequence so far: 21 23 25 27 3
15th char = '3'
Sequence so far: 21 23 25 27 31
16th char = '1'
Sequence so far: 21 23 25 27 31 
17th char = ' '
Sequence so far: 21 23 25 27 31 3
18th char = '3'
Sequence so far: 21 23 25 27 31 33
19th char = '3'
Sequence so far: 21 23 25 27 31 33 
20th char = ' '


In [99]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 10)

7
Sequence so far: 21 23 25 
11th char = ' '
Sequence so far: 21 23 25 2
12th char = '2'
Sequence so far: 21 23 25 27
13th char = '7'
Sequence so far: 21 23 25 27 
14th char = ' '
Sequence so far: 21 23 25 27 2
15th char = '2'
Sequence so far: 21 23 25 27 29
16th char = '9'
Sequence so far: 21 23 25 27 29 
17th char = ' '
Sequence so far: 21 23 25 27 29 3
18th char = '3'
Sequence so far: 21 23 25 27 29 31
19th char = '1'
Sequence so far: 21 23 25 27 29 31 
20th char = ' '


## top 40 from 2 4 6 circ

In [ ]:
# from 2 4 6

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
# head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = ([(0, 13), (1, 8), (0, 15), (0, 14), (4, 3), (16, 0), (15, 25), (5, 25), (5, 26), (7, 30), (1, 11), (8, 0), (22, 25), (6, 11), (5, 29), (2, 2), (6, 26), (6, 24), (5, 15), (20, 17), (6, 5), (5, 17), (2, 30), (7, 9), (4, 0), (13, 6), (5, 11), (0, 21), (0, 7), (29, 1), (0, 1), (29, 5), (8, 4), (5, 16), (31, 4), (18, 19), (28, 16), (18, 9), (0, 4), (4, 16)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 21 23 25 
11th char = ' '
Sequence so far: 21 23 25 2
12th char = '2'
Sequence so far: 21 23 25 25
13th char = '5'


# 3 6 9

In [107]:
clean_text = "3 6 9"
corr_text = "5 3 9"
# corr_text = "1 2 3"

clean

In [108]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
mlps_not_ablate = [layer for layer in range(32)]
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 3 6 9 
8th char = ' '
Sequence so far: 3 6 9 1
9th char = '1'
Sequence so far: 3 6 9 12
10th char = '2'


## top 50

In [112]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 3 6 9 
8th char = ' '
Sequence so far: 3 6 9 1
9th char = '1'
Sequence so far: 3 6 9 10
10th char = '0'


In [113]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:20]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 3 6 9 
8th char = ' '
Sequence so far: 3 6 9 1
9th char = '1'
Sequence so far: 3 6 9 10
10th char = '0'


In [114]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:30]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 3 6 9 
8th char = ' '
Sequence so far: 3 6 9 1
9th char = '1'
Sequence so far: 3 6 9 10
10th char = '0'


In [115]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:40]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 3 6 9 
8th char = ' '
Sequence so far: 3 6 9 1
9th char = '1'
Sequence so far: 3 6 9 10
10th char = '0'


In [116]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[30:40]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

7
Sequence so far: 3 6 9 
8th char = ' '
Sequence so far: 3 6 9 1
9th char = '1'
Sequence so far: 3 6 9 12
10th char = '2'


## random

In [117]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 5)

7
Sequence so far: 3 6 9 
8th char = ' '
Sequence so far: 3 6 9 1
9th char = '1'
Sequence so far: 3 6 9 12
10th char = '2'
Sequence so far: 3 6 9 12 
11th char = ' '
Sequence so far: 3 6 9 12 1
12th char = '1'


In [118]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 5)

7
Sequence so far: 3 6 9 
8th char = ' '
Sequence so far: 3 6 9 1
9th char = '1'
Sequence so far: 3 6 9 12
10th char = '2'
Sequence so far: 3 6 9 12 
11th char = ' '
Sequence so far: 3 6 9 12 1
12th char = '1'


In [119]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 5)

7
Sequence so far: 3 6 9 
8th char = ' '
Sequence so far: 3 6 9 1
9th char = '1'
Sequence so far: 3 6 9 12
10th char = '2'
Sequence so far: 3 6 9 12 
11th char = ' '
Sequence so far: 3 6 9 12 1
12th char = '1'


## top 40 from 2 4 6 circ

In [120]:
# from 2 4 6

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
# head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = ([(0, 13), (1, 8), (0, 15), (0, 14), (4, 3), (16, 0), (15, 25), (5, 25), (5, 26), (7, 30), (1, 11), (8, 0), (22, 25), (6, 11), (5, 29), (2, 2), (6, 26), (6, 24), (5, 15), (20, 17), (6, 5), (5, 17), (2, 30), (7, 9), (4, 0), (13, 6), (5, 11), (0, 21), (0, 7), (29, 1), (0, 1), (29, 5), (8, 4), (5, 16), (31, 4), (18, 19), (28, 16), (18, 9), (0, 4), (4, 16)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 7)

7
Sequence so far: 3 6 9 
8th char = ' '
Sequence so far: 3 6 9 1
9th char = '1'
Sequence so far: 3 6 9 10
10th char = '0'
Sequence so far: 3 6 9 10 
11th char = ' '
Sequence so far: 3 6 9 10 1
12th char = '1'
Sequence so far: 3 6 9 10 10
13th char = '0'
Sequence so far: 3 6 9 10 10 
14th char = ' '


# 100 200 300

In [132]:
clean_text = "100 200 300"
corr_text = "5 3 9"
# corr_text = "1 2 3"

clean

In [133]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
mlps_not_ablate = [layer for layer in range(32)]
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

7
Sequence so far: 100 200 300 
14th char = ' '
Sequence so far: 100 200 300 4
15th char = '4'
Sequence so far: 100 200 300 40
16th char = '0'
Sequence so far: 100 200 300 400
17th char = '0'
Sequence so far: 100 200 300 400 
18th char = ' '
Sequence so far: 100 200 300 400 5
19th char = '5'


## top 50

In [134]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

7
Sequence so far: 100 200 300 
14th char = ' '
Sequence so far: 100 200 300 3
15th char = '3'
Sequence so far: 100 200 300 30
16th char = '0'
Sequence so far: 100 200 300 300
17th char = '0'
Sequence so far: 100 200 300 300 
18th char = ' '
Sequence so far: 100 200 300 300 3
19th char = '3'


In [135]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:20]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

7
Sequence so far: 100 200 300 
14th char = ' '
Sequence so far: 100 200 300 4
15th char = '4'
Sequence so far: 100 200 300 40
16th char = '0'
Sequence so far: 100 200 300 400
17th char = '0'
Sequence so far: 100 200 300 400 
18th char = ' '
Sequence so far: 100 200 300 400 5
19th char = '5'


In [136]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:30]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

7
Sequence so far: 100 200 300 
14th char = ' '
Sequence so far: 100 200 300 4
15th char = '4'
Sequence so far: 100 200 300 40
16th char = '0'
Sequence so far: 100 200 300 400
17th char = '0'
Sequence so far: 100 200 300 400 
18th char = ' '
Sequence so far: 100 200 300 400 5
19th char = '5'


In [137]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:40]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

7
Sequence so far: 100 200 300 
14th char = ' '
Sequence so far: 100 200 300 4
15th char = '4'
Sequence so far: 100 200 300 40
16th char = '0'
Sequence so far: 100 200 300 400
17th char = '0'
Sequence so far: 100 200 300 400 
18th char = ' '
Sequence so far: 100 200 300 400 5
19th char = '5'


In [138]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[30:40]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

7
Sequence so far: 100 200 300 
14th char = ' '
Sequence so far: 100 200 300 4
15th char = '4'
Sequence so far: 100 200 300 40
16th char = '0'
Sequence so far: 100 200 300 400
17th char = '0'
Sequence so far: 100 200 300 400 
18th char = ' '
Sequence so far: 100 200 300 400 5
19th char = '5'


## random

In [139]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

7
Sequence so far: 100 200 300 
14th char = ' '
Sequence so far: 100 200 300 4
15th char = '4'
Sequence so far: 100 200 300 40
16th char = '0'
Sequence so far: 100 200 300 400
17th char = '0'
Sequence so far: 100 200 300 400 
18th char = ' '
Sequence so far: 100 200 300 400 5
19th char = '5'


In [140]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

7
Sequence so far: 100 200 300 
14th char = ' '
Sequence so far: 100 200 300 4
15th char = '4'
Sequence so far: 100 200 300 40
16th char = '0'
Sequence so far: 100 200 300 400
17th char = '0'
Sequence so far: 100 200 300 400 
18th char = ' '
Sequence so far: 100 200 300 400 5
19th char = '5'


In [141]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

7
Sequence so far: 100 200 300 
14th char = ' '
Sequence so far: 100 200 300 4
15th char = '4'
Sequence so far: 100 200 300 40
16th char = '0'
Sequence so far: 100 200 300 400
17th char = '0'
Sequence so far: 100 200 300 400 
18th char = ' '
Sequence so far: 100 200 300 400 5
19th char = '5'


## top 40 from 2 4 6 circ

In [142]:
# from 2 4 6

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
# head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = ([(0, 13), (1, 8), (0, 15), (0, 14), (4, 3), (16, 0), (15, 25), (5, 25), (5, 26), (7, 30), (1, 11), (8, 0), (22, 25), (6, 11), (5, 29), (2, 2), (6, 26), (6, 24), (5, 15), (20, 17), (6, 5), (5, 17), (2, 30), (7, 9), (4, 0), (13, 6), (5, 11), (0, 21), (0, 7), (29, 1), (0, 1), (29, 5), (8, 4), (5, 16), (31, 4), (18, 19), (28, 16), (18, 9), (0, 4), (4, 16)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 7)

7
Sequence so far: 100 200 300 
14th char = ' '
Sequence so far: 100 200 300 4
15th char = '4'
Sequence so far: 100 200 300 40
16th char = '0'
Sequence so far: 100 200 300 400
17th char = '0'
Sequence so far: 100 200 300 400 
18th char = ' '
Sequence so far: 100 200 300 400 5
19th char = '5'
Sequence so far: 100 200 300 400 50
20th char = '0'


# uno dos tres

In [143]:
clean_text = "uno dos tres"
corr_text = "uno uno uno" # dos tres cinco seis

clean

In [164]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
mlps_not_ablate = [layer for layer in range(32)]
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 10)

4
Sequence so far: uno dos trescuatro
5th char = 'cuatro'
Sequence so far: uno dos trescuatrocinco
6th char = 'cinco'
Sequence so far: uno dos trescuatrocincoseis
7th char = 'seis'
Sequence so far: uno dos trescuatrocincoseisseven
8th char = 'seven'
Sequence so far: uno dos trescuatrocincoseisseveneight
9th char = 'eight'
Sequence so far: uno dos trescuatrocincoseisseveneightnine
10th char = 'nine'
Sequence so far: uno dos trescuatrocincoseisseveneightnineten
11th char = 'ten'
Sequence so far: uno dos trescuatrocincoseisseveneightnineteneleven
12th char = 'eleven'
Sequence so far: uno dos trescuatrocincoseisseveneightnineteneleventwelve
13th char = 'twelve'
Sequence so far: uno dos trescuatrocincoseisseveneightnineteneleventwelveth
14th char = 'th'


In [145]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(0, 13)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

4
Sequence so far: uno dos trescuatro
5th char = 'cuatro'
Sequence so far: uno dos trescuatrocinco
6th char = 'cinco'
Sequence so far: uno dos trescuatrocincoseis
7th char = 'seis'


In [146]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

4
Sequence so far: uno dos tres<0x0A>
5th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A>
6th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A>
7th char = '<0x0A>'


## top 50

In [147]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos tres<0x0A>
5th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A>
6th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A>o
7th char = 'o'
Sequence so far: uno dos tres<0x0A><0x0A>otres
8th char = 'tres'
Sequence so far: uno dos tres<0x0A><0x0A>otresuno
9th char = 'uno'
Sequence so far: uno dos tres<0x0A><0x0A>otresuno<0x0A>
10th char = '<0x0A>'


In [148]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:20]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos tres<0x0A>
5th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A>
6th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A>
7th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A>
8th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A><0x0A>
9th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A><0x0A><0x0A>
10th char = '<0x0A>'


In [149]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:30]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos tres<0x0A>
5th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A>
6th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A>
7th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A>
8th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A><0x0A>
9th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A><0x0A>uno
10th char = 'uno'


In [150]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[:40]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos tres<0x0A>
5th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A>
6th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A>o
7th char = 'o'
Sequence so far: uno dos tres<0x0A><0x0A>ocho
8th char = 'cho'
Sequence so far: uno dos tres<0x0A><0x0A>ochodos
9th char = 'dos'
Sequence so far: uno dos tres<0x0A><0x0A>ochodosdos
10th char = 'dos'


In [151]:
# from 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = head_to_remove[30:40]
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos trescuatro
5th char = 'cuatro'
Sequence so far: uno dos trescuatrofive
6th char = 'five'
Sequence so far: uno dos trescuatrofivesix
7th char = 'six'
Sequence so far: uno dos trescuatrofivesixseven
8th char = 'seven'
Sequence so far: uno dos trescuatrofivesixseveneight
9th char = 'eight'
Sequence so far: uno dos trescuatrofivesixseveneightnine
10th char = 'nine'


## random

In [152]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos tres"
5th char = '"'
Sequence so far: uno dos tres"<0x0A>
6th char = '<0x0A>'
Sequence so far: uno dos tres"<0x0A><0x0A>
7th char = '<0x0A>'
Sequence so far: uno dos tres"<0x0A><0x0A>Answer
8th char = 'Answer'
Sequence so far: uno dos tres"<0x0A><0x0A>Answer:
9th char = ':'
Sequence so far: uno dos tres"<0x0A><0x0A>Answer:<0x0A>
10th char = '<0x0A>'


In [153]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos tres<0x0A>
5th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A>
6th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A>
7th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A>
8th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A><0x0A>
9th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A><0x0A><0x0A>
10th char = '<0x0A>'


In [154]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 50)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos tres<0x0A>
5th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A>
6th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A>
7th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A>
8th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A><0x0A>
9th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A><0x0A><0x0A>
10th char = '<0x0A>'


## random 1

In [156]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 1)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos trescuatro
5th char = 'cuatro'
Sequence so far: uno dos trescuatrocinco
6th char = 'cinco'
Sequence so far: uno dos trescuatrocinco<0x0A>
7th char = '<0x0A>'
Sequence so far: uno dos trescuatrocinco<0x0A><0x0A>
8th char = '<0x0A>'
Sequence so far: uno dos trescuatrocinco<0x0A><0x0A><0x0A>
9th char = '<0x0A>'
Sequence so far: uno dos trescuatrocinco<0x0A><0x0A><0x0A><0x0A>
10th char = '<0x0A>'


In [159]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 1)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos trescuatro
5th char = 'cuatro'
Sequence so far: uno dos trescuatrocinco
6th char = 'cinco'
Sequence so far: uno dos trescuatrocincosix
7th char = 'six'
Sequence so far: uno dos trescuatrocincosixseven
8th char = 'seven'
Sequence so far: uno dos trescuatrocincosixseveneight
9th char = 'eight'
Sequence so far: uno dos trescuatrocincosixseveneightnine
10th char = 'nine'


## random 10

In [157]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 10)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos trescuatro
5th char = 'cuatro'
Sequence so far: uno dos trescuatrocinco
6th char = 'cinco'
Sequence so far: uno dos trescuatrocincosix
7th char = 'six'
Sequence so far: uno dos trescuatrocincosixseven
8th char = 'seven'
Sequence so far: uno dos trescuatrocincosixseveneight
9th char = 'eight'
Sequence so far: uno dos trescuatrocincosixseveneightnine
10th char = 'nine'


In [165]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 10)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos tres<0x0A>
5th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A>
6th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A>
7th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A>
8th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A><0x0A>
9th char = '<0x0A>'
Sequence so far: uno dos tres<0x0A><0x0A><0x0A><0x0A><0x0A><0x0A>
10th char = '<0x0A>'


In [167]:
head_to_remove

[(17, 27),
 (16, 15),
 (4, 16),
 (16, 25),
 (0, 23),
 (18, 10),
 (27, 3),
 (9, 6),
 (9, 7),
 (14, 8)]

In [168]:
top50_1234 = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
overlap = list(set(top50_1234) & set(head_to_remove))
overlap

[(14, 8)]

In [170]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(14, 8)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 6)

4
Sequence so far: uno dos trescuatro
5th char = 'cuatro'
Sequence so far: uno dos trescuatrocinco
6th char = 'cinco'
Sequence so far: uno dos trescuatrocinco<0x0A>
7th char = '<0x0A>'
Sequence so far: uno dos trescuatrocinco<0x0A><0x0A>
8th char = '<0x0A>'
Sequence so far: uno dos trescuatrocinco<0x0A><0x0A><0x0A>
9th char = '<0x0A>'
Sequence so far: uno dos trescuatrocinco<0x0A><0x0A><0x0A><0x0A>
10th char = '<0x0A>'


## top 40 from 2 4 6 circ

In [155]:
# from 2 4 6

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
# head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove = ([(0, 13), (1, 8), (0, 15), (0, 14), (4, 3), (16, 0), (15, 25), (5, 25), (5, 26), (7, 30), (1, 11), (8, 0), (22, 25), (6, 11), (5, 29), (2, 2), (6, 26), (6, 24), (5, 15), (20, 17), (6, 5), (5, 17), (2, 30), (7, 9), (4, 0), (13, 6), (5, 11), (0, 21), (0, 7), (29, 1), (0, 1), (29, 5), (8, 4), (5, 16), (31, 4), (18, 19), (28, 16), (18, 9), (0, 4), (4, 16)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)]

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 7)

4
Sequence so far: uno dos tres,
5th char = ','
Sequence so far: uno dos tres,and
6th char = 'and'
Sequence so far: uno dos tres,andthe
7th char = 'the'
Sequence so far: uno dos tres,andtheother
8th char = 'other'
Sequence so far: uno dos tres,andtheothertwo
9th char = 'two'
Sequence so far: uno dos tres,andtheothertwoare
10th char = 'are'
Sequence so far: uno dos tres,andtheothertwoaretwo
11th char = 'two'


# "What comes after Monday is Tuesday, and two days after is"

In [62]:
clean_text = "What comes after Monday is Tuesday, and two days after is"
corr_text = "What comes after X is Y, and two days after is"

clean

In [63]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
mlps_not_ablate = [layer for layer in range(32)]
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

13
Sequence so far: What comes after Monday is Tuesday, and two days after isTh
17th char = 'Th'
Sequence so far: What comes after Monday is Tuesday, and two days after isThurs
18th char = 'urs'
Sequence so far: What comes after Monday is Tuesday, and two days after isThursday
19th char = 'day'


In [64]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(0, 13)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

13
Sequence so far: What comes after Monday is Tuesday, and two days after isTh
17th char = 'Th'
Sequence so far: What comes after Monday is Tuesday, and two days after isThurs
18th char = 'urs'
Sequence so far: What comes after Monday is Tuesday, and two days after isThursday
19th char = 'day'


In [65]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

13
Sequence so far: What comes after Monday is Tuesday, and two days after isTh
17th char = 'Th'
Sequence so far: What comes after Monday is Tuesday, and two days after isThurs
18th char = 'urs'
Sequence so far: What comes after Monday is Tuesday, and two days after isThursday
19th char = 'day'


Obtain heads from top 20 of https://colab.research.google.com/drive/1p_x98vp4OMx46rphUdIk64E7P94cQmg9#scrollTo=susSZdqpqVzd&line=1&uniqifier=1

In [66]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25),
    (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15),
    (26, 2), (10, 25), (2, 2), (23, 2)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 3)

13
Sequence so far: What comes after Monday is Tuesday, and two days after isTh
17th char = 'Th'
Sequence so far: What comes after Monday is Tuesday, and two days after isThurs
18th char = 'urs'
Sequence so far: What comes after Monday is Tuesday, and two days after isThursday
19th char = 'day'


# What are the months in a year?

In [47]:
clean_text = "What are the months in a year?"
corr_text = "What are the X in a year?"

clean

In [34]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
mlps_not_ablate = [layer for layer in range(32)]
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 30)

9
Sequence so far: What are the months in a year?<0x0A>
10th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>
11th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>There
12th char = 'There'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare
13th char = 'are'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 
14th char = ' '
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 1
15th char = '1'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12
16th char = '2'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12months
17th char = 'months'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsin
18th char = 'in'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsina
19th char = 'a'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsinayear
20th char = 'year'
Sequence so far: Wha

In [35]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(23, 17)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 30)

9
Sequence so far: What are the months in a year?<0x0A>
10th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>
11th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>There
12th char = 'There'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare
13th char = 'are'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 
14th char = ' '
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 1
15th char = '1'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12
16th char = '2'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12months
17th char = 'months'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsin
18th char = 'in'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsina
19th char = 'a'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsinayear
20th char = 'year'
Sequence so far: Wha

In [36]:
heads_not_ablate = []

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 30)

9
Sequence so far: What are the months in a year?That
10th char = 'That'
Sequence so far: What are the months in a year?That,
11th char = ','
Sequence so far: What are the months in a year?That,g
12th char = 'g'
Sequence so far: What are the months in a year?That,g.,
13th char = '.,'
Sequence so far: What are the months in a year?That,g.,a
14th char = 'a'
Sequence so far: What are the months in a year?That,g.,aand
15th char = 'and'
Sequence so far: What are the months in a year?That,g.,aand 
16th char = ' '
Sequence so far: What are the months in a year?That,g.,aand 1
17th char = '1'
Sequence so far: What are the months in a year?That,g.,aand 1(
18th char = '('
Sequence so far: What are the months in a year?That,g.,aand 1(As
19th char = 'As'
Sequence so far: What are the months in a year?That,g.,aand 1(As.
20th char = '.'
Sequence so far: What are the months in a year?That,g.,aand 1(As.<0x0A>
21th char = '<0x0A>'
Sequence so far: What are the months in a year?That,g.,aand 1(As.<0x0A>al

In [37]:
# top 7 heads from seqcont months

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(23, 17), (17, 11), (16, 0), (26, 14), (18, 9), (5, 25), (22, 20)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 30)

9
Sequence so far: What are the months in a year?<0x0A>
10th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>
11th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>There
12th char = 'There'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare
13th char = 'are'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 
14th char = ' '
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 1
15th char = '1'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12
16th char = '2'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12months
17th char = 'months'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsin
18th char = 'in'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsina
19th char = 'a'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsinayear
20th char = 'year'
Sequence so far: Wha

## top 50 from seqcont circs

In [48]:
# top 50 heads from seqcont months

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(23, 17), (17, 11), (16, 0), (26, 14), (18, 9), (5, 25), (22, 20), (6, 24), (26, 9), (12, 18), (13, 20), (19, 12), (27, 29), (13, 14), (16, 14), (12, 26), (19, 30), (16, 18), (31, 27), (26, 28), (16, 1), (18, 1), (19, 28), (18, 31), (29, 4), (17, 0), (14, 1), (17, 12), (12, 15), (28, 16), (10, 1), (16, 19), (9, 27), (30, 1), (19, 27), (0, 3), (15, 11), (21, 3), (11, 19), (12, 0), (23, 11), (8, 14), (16, 8), (22, 13), (13, 3), (4, 19), (14, 15), (12, 20), (19, 16), (18, 5)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 40)

9
Sequence so far: What are the months in a year?<0x0A>
10th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>
11th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>There
12th char = 'There'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare
13th char = 'are'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 
14th char = ' '
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 1
15th char = '1'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12
16th char = '2'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12months
17th char = 'months'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsin
18th char = 'in'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsina
19th char = 'a'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsinayear
20th char = 'year'
Sequence so far: Wha

In [49]:
# top 50 heads from seqcont 1234

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 40)

9
Sequence so far: What are the months in a year?<0x0A>
10th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>
11th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>There
12th char = 'There'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare
13th char = 'are'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 
14th char = ' '
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 1
15th char = '1'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12
16th char = '2'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 124
17th char = '4'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 124months
18th char = 'months'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 124monthsin
19th char = 'in'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 124monthsina
20th char = 'a'
Sequence so far: What are the mo

In [65]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove_num = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove_months = ([(23, 17), (17, 11), (16, 0), (26, 14), (18, 9), (5, 25), (22, 20), (6, 24), (26, 9), (12, 18), (13, 20), (19, 12), (27, 29), (13, 14), (16, 14), (12, 26), (19, 30), (16, 18), (31, 27), (26, 28), (16, 1), (18, 1), (19, 28), (18, 31), (29, 4), (17, 0), (14, 1), (17, 12), (12, 15), (28, 16), (10, 1), (16, 19), (9, 27), (30, 1), (19, 27), (0, 3), (15, 11), (21, 3), (11, 19), (12, 0), (23, 11), (8, 14), (16, 8), (22, 13), (13, 3), (4, 19), (14, 15), (12, 20), (19, 16), (18, 5)])
head_to_remove = head_to_remove_num + head_to_remove_months
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 60)

9
Sequence so far: What are the months in a year?<0x0A>
10th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>
11th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A><0x0A>
12th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A><0x0A>What
13th char = 'What'
Sequence so far: What are the months in a year?<0x0A><0x0A><0x0A>Whatare
14th char = 'are'
Sequence so far: What are the months in a year?<0x0A><0x0A><0x0A>Whatarethe
15th char = 'the'
Sequence so far: What are the months in a year?<0x0A><0x0A><0x0A>Whatarethemonths
16th char = 'months'
Sequence so far: What are the months in a year?<0x0A><0x0A><0x0A>Whatarethemonthsin
17th char = 'in'
Sequence so far: What are the months in a year?<0x0A><0x0A><0x0A>Whatarethemonthsina
18th char = 'a'
Sequence so far: What are the months in a year?<0x0A><0x0A><0x0A>Whatarethemonthsinayear
19th char = 'year'
Sequence so far: What are the months in a year?<0x0A><0x0A><0x0A>Wha

In [67]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove_num = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove_months = ([(23, 17), (17, 11), (16, 0), (26, 14), (18, 9), (5, 25), (22, 20), (6, 24), (26, 9), (12, 18), (13, 20), (19, 12), (27, 29), (13, 14), (16, 14), (12, 26), (19, 30), (16, 18), (31, 27), (26, 28), (16, 1), (18, 1), (19, 28), (18, 31), (29, 4), (17, 0), (14, 1), (17, 12), (12, 15), (28, 16), (10, 1), (16, 19), (9, 27), (30, 1), (19, 27), (0, 3), (15, 11), (21, 3), (11, 19), (12, 0), (23, 11), (8, 14), (16, 8), (22, 13), (13, 3), (4, 19), (14, 15), (12, 20), (19, 16), (18, 5)])
head_to_remove_num = head_to_remove_num[:10]
head_to_remove_months = head_to_remove_months[:10]
head_to_remove = head_to_remove_num + head_to_remove_months
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 30)

9
Sequence so far: What are the months in a year?<0x0A>
10th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>
11th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>What
12th char = 'What'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatare
13th char = 'are'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethe
14th char = 'the'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethemonths
15th char = 'months'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethemonthsin
16th char = 'in'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethemonthsina
17th char = 'a'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethemonthsinayear
18th char = 'year'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethemonthsinayear?
19th char = '?'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethemonthsinayear?<0x0A>
20th ch

## top 20

In [69]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove_num = ([(20, 17), (1, 11), (0, 30), (0, 9), (5, 26), (16, 0), (13, 6), (15, 25), (5, 15), (6, 11), (5, 25), (5, 17), (1, 28), (29, 5), (4, 3), (15, 15), (26, 2), (10, 25), (2, 2), (23, 2), (30, 13), (25, 23), (6, 24), (11, 28), (10, 1), (7, 0), (18, 19), (0, 26), (9, 26), (18, 9), (19, 28), (10, 5), (2, 24), (15, 26), (31, 4), (1, 16), (11, 27), (31, 11), (16, 14), (23, 17), (23, 27), (14, 8), (1, 22), (23, 6), (7, 30), (19, 30), (3, 6), (19, 12), (20, 25), (17, 17)])
head_to_remove_months = ([(23, 17), (17, 11), (16, 0), (26, 14), (18, 9), (5, 25), (22, 20), (6, 24), (26, 9), (12, 18), (13, 20), (19, 12), (27, 29), (13, 14), (16, 14), (12, 26), (19, 30), (16, 18), (31, 27), (26, 28), (16, 1), (18, 1), (19, 28), (18, 31), (29, 4), (17, 0), (14, 1), (17, 12), (12, 15), (28, 16), (10, 1), (16, 19), (9, 27), (30, 1), (19, 27), (0, 3), (15, 11), (21, 3), (11, 19), (12, 0), (23, 11), (8, 14), (16, 8), (22, 13), (13, 3), (4, 19), (14, 15), (12, 20), (19, 16), (18, 5)])
head_to_remove_num = head_to_remove_num[:20]
head_to_remove_months = head_to_remove_months[:20]
head_to_remove = head_to_remove_num + head_to_remove_months
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 50)

9
Sequence so far: What are the months in a year?<0x0A>
10th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>
11th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>What
12th char = 'What'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatare
13th char = 'are'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethe
14th char = 'the'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethemonths
15th char = 'months'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethemonthsin
16th char = 'in'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethemonthsina
17th char = 'a'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethemonthsinayear
18th char = 'year'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethemonthsinayear?
19th char = '?'
Sequence so far: What are the months in a year?<0x0A><0x0A>Whatarethemonthsinayear?<0x0A>
20th ch

## random

In [70]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 40)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 40)

9
Sequence so far: What are the months in a year?<0x0A>
10th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A>There
11th char = 'There'
Sequence so far: What are the months in a year?<0x0A>Thereare
12th char = 'are'
Sequence so far: What are the months in a year?<0x0A>Thereare 
13th char = ' '
Sequence so far: What are the months in a year?<0x0A>Thereare 1
14th char = '1'
Sequence so far: What are the months in a year?<0x0A>Thereare 12
15th char = '2'
Sequence so far: What are the months in a year?<0x0A>Thereare 12months
16th char = 'months'
Sequence so far: What are the months in a year?<0x0A>Thereare 12monthsin
17th char = 'in'
Sequence so far: What are the months in a year?<0x0A>Thereare 12monthsina
18th char = 'a'
Sequence so far: What are the months in a year?<0x0A>Thereare 12monthsinayear
19th char = 'year'
Sequence so far: What are the months in a year?<0x0A>Thereare 12monthsinayear.
20th char = '.'
Sequence so far: What are the months in a year?<0x0A>Therear

In [71]:
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]
head_to_remove = random.sample(all_possible_pairs, 40)
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 40)

9
Sequence so far: What are the months in a year?<0x0A>
10th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>
11th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>There
12th char = 'There'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare
13th char = 'are'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 
14th char = ' '
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 1
15th char = '1'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12
16th char = '2'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12months
17th char = 'months'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsin
18th char = 'in'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsina
19th char = 'a'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsinayear
20th char = 'year'
Sequence so far: Wha

### 100 rand

In [57]:
import random

# Generate all possible pairs (i, j) where i and j range from 0 to 31 inclusive
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]

# Randomly choose 100 pairs from the list
head_to_remove = random.sample(all_possible_pairs, 100)

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 40)

9
Sequence so far: What are the months in a year?<0x0A>
10th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>
11th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>Answer
12th char = 'Answer'
Sequence so far: What are the months in a year?<0x0A><0x0A>Answer:
13th char = ':'
Sequence so far: What are the months in a year?<0x0A><0x0A>Answer:The
14th char = 'The'
Sequence so far: What are the months in a year?<0x0A><0x0A>Answer:Themonths
15th char = 'months'
Sequence so far: What are the months in a year?<0x0A><0x0A>Answer:Themonthsin
16th char = 'in'
Sequence so far: What are the months in a year?<0x0A><0x0A>Answer:Themonthsina
17th char = 'a'
Sequence so far: What are the months in a year?<0x0A><0x0A>Answer:Themonthsinayear
18th char = 'year'
Sequence so far: What are the months in a year?<0x0A><0x0A>Answer:Themonthsinayearare
19th char = 'are'
Sequence so far: What are the months in a year?<0x0A><0x0A>Answer:ThemonthsinayearareJanuar

In [64]:
all_possible_pairs = [(i, j) for i in range(32) for j in range(32)]
head_to_remove = random.sample(all_possible_pairs, 100)
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 40)

9
Sequence so far: What are the months in a year?<0x0A>
10th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>
11th char = '<0x0A>'
Sequence so far: What are the months in a year?<0x0A><0x0A>There
12th char = 'There'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare
13th char = 'are'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 
14th char = ' '
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 1
15th char = '1'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12
16th char = '2'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12months
17th char = 'months'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsin
18th char = 'in'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsina
19th char = 'a'
Sequence so far: What are the months in a year?<0x0A><0x0A>Thereare 12monthsinayear
20th char = 'year'
Sequence so far: Wha

In [63]:
len(heads_not_ablate)

924

# jan feb mar

In [41]:
clean_text = "January February March"
corr_text = "July April September"

clean

In [42]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
mlps_not_ablate = [layer for layer in range(32)]
ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 2)

4
Sequence so far: January February MarchApril
5th char = 'April'
Sequence so far: January February MarchAprilMay
6th char = 'May'


In [43]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(23, 17)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 2)

4
Sequence so far: January February MarchApril
5th char = 'April'
Sequence so far: January February MarchAprilMay
6th char = 'May'


In [46]:
# top 7 heads from seqcont months

heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(23, 17), (17, 11), (16, 0), (26, 14), (18, 9), (5, 25), (22, 20)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 2)

4
Sequence so far: January February MarchApril
5th char = 'April'
Sequence so far: January February MarchAprilMay
6th char = 'May'


In [44]:
heads_not_ablate = [(layer, head) for layer in range(32) for head in range(32)]  # unablated
head_to_remove = ([(23, 17), (17, 11), (16, 0), (26, 14), (18, 9), (5, 25), (22, 20), (6, 24), (26, 9), (12, 18), (13, 20), (19, 12), (27, 29), (13, 14), (16, 14), (12, 26), (19, 30), (16, 18), (31, 27), (26, 28), (16, 1), (18, 1), (19, 28), (18, 31), (29, 4), (17, 0), (14, 1), (17, 12), (12, 15), (28, 16), (10, 1), (16, 19), (9, 27), (30, 1), (19, 27), (0, 3), (15, 11), (21, 3), (11, 19), (12, 0), (23, 11), (8, 14), (16, 8), (22, 13), (13, 3), (4, 19), (14, 15), (12, 20), (19, 16), (18, 5)])
heads_not_ablate = [x for x in heads_not_ablate if (x not in head_to_remove)]

mlps_not_ablate = [layer for layer in range(32)] #  if layer <10

ablate_then_gen(model, clean_text, corr_text, heads_not_ablate, mlps_not_ablate, 30)

4
Sequence so far: January February March 
5th char = ' '
Sequence so far: January February March 2
6th char = '2'
Sequence so far: January February March 20
7th char = '0'
Sequence so far: January February March 202
8th char = '2'
Sequence so far: January February March 2023
9th char = '3'
Sequence so far: January February March 2023<0x0A>
10th char = '<0x0A>'
Sequence so far: January February March 2023<0x0A><0x0A>
11th char = '<0x0A>'
Sequence so far: January February March 2023<0x0A><0x0A>J
12th char = 'J'
Sequence so far: January February March 2023<0x0A><0x0A>Janu
13th char = 'anu'
Sequence so far: January February March 2023<0x0A><0x0A>January
14th char = 'ary'
Sequence so far: January February March 2023<0x0A><0x0A>January 
15th char = ' '
Sequence so far: January February March 2023<0x0A><0x0A>January 2
16th char = '2'
Sequence so far: January February March 2023<0x0A><0x0A>January 20
17th char = '0'
Sequence so far: January February March 2023<0x0A><0x0A>January 202
18th char